# PySAL with Numba Performance Test

In [1]:
import sys
import libpysal as lps
import numpy as np
import geopandas as gpd
import esda
import time

## Testing method

In [2]:


"""
Usage: 
python3 perf_pysal.py FILE_PATH VARIABLE_NAME PERMUTATIONS CPU_THREADS
python3 perf_pysal.py ./data/natregimes.shp HR60 999 1

data_path = sys.argv[1]
var_name = sys.argv[2]
perms = int(sys.argv[3])
cpu_threads = int(sys.argv[4])
"""

def test(data_path, var_name, perms, cpu_threads):
    np.random.seed(12345)


    # prepare the data and weights
    gdf = gpd.read_file(data_path)
    x = gdf[var_name]


    if data_path == './data/Chicago_parcels_points.shp':
        w = lps.weights.KNN.from_dataframe(gdf, k=10)
    else:
        w = lps.weights.Queen.from_dataframe(gdf)
        # remove islands
        if len(w.islands) > 0:
            df_noi = gdf[~gdf.index.isin(w.islands)]
            w = lps.weights.w_subset(w, df_noi.index.tolist())
            x = df_noi[var_name]
    w.transform = 'r'


    # function to execute and time
    start_time = time.time()
    li = esda.moran.Moran_Local(x, w, permutations=perms, n_jobs=cpu_threads)
    run_time = time.time() - start_time

    print("{0} {1} {2}: {3} seconds".format(data_path, perms, cpu_threads, run_time))
    return run_time

## Data

In [3]:
test_ds = {'./data/natregimes.shp' : 'HR60',
          './data/us-sdoh-2014.shp' : 'ep_unem',
          './data/NYC_Area2010_2data.shp' : 'CE01_02',
          #'./data/Chicago_parcels_points.shp' : 'EstBuild'
          }

perm_set = [999, 9999, 99999]

threads_set = [1, 8, 16]

## Test

In [4]:
try:
    import numba
    print(f"Numba version installed: {numba.__version__}")
except:
    print("Numba not installed")

Numba version installed: 0.52.0


In [5]:
for ds_path, var_name in test_ds.items():
    print(ds_path, var_name)
    for perm in perm_set:
        for threads in threads_set:
            times = []
            j = 5
            for i in range(j+1):
                t = test(ds_path, var_name, perm, threads)
                if i > 0:
                    times.append(t)
            print(f"\tAverage of {j}: {np.mean(times)}")

./data/natregimes.shp HR60
./data/natregimes.shp 999 1: 4.192052125930786 seconds
./data/natregimes.shp 999 1: 0.17628169059753418 seconds
./data/natregimes.shp 999 1: 0.17741703987121582 seconds
./data/natregimes.shp 999 1: 0.17043399810791016 seconds
./data/natregimes.shp 999 1: 0.1668100357055664 seconds
./data/natregimes.shp 999 1: 0.16829395294189453 seconds
	Average of 5: 0.17184734344482422
./data/natregimes.shp 999 8: 6.920797109603882 seconds
./data/natregimes.shp 999 8: 0.22967314720153809 seconds
./data/natregimes.shp 999 8: 0.25632500648498535 seconds
./data/natregimes.shp 999 8: 0.21572494506835938 seconds
./data/natregimes.shp 999 8: 0.23557615280151367 seconds
./data/natregimes.shp 999 8: 0.23087787628173828 seconds
	Average of 5: 0.23363542556762695
./data/natregimes.shp 999 16: 0.45927977561950684 seconds
./data/natregimes.shp 999 16: 5.089677095413208 seconds
./data/natregimes.shp 999 16: 5.43870997428894 seconds
./data/natregimes.shp 999 16: 5.00357723236084 seconds


KeyboardInterrupt: 